In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
from flask import Flask, request, jsonify

In [ ]:
data = pd.read_csv("diabetes.csv")

In [ ]:
data.columns

Index(['Glucose', 'Insulin', 'BMI', 'Age', 'Outcome'], dtype='object')

In [ ]:
data.head()

,Glucose,Insulin,BMI,Age,Outcome
0,148,0,33.6,50,1
1,85,0,26.6,31,0
2,183,0,23.3,32,1
3,89,94,28.1,21,0
4,137,168,43.1,33,1


In [ ]:
data.describe()

,Glucose,Insulin,BMI,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000
mean,120.894531,79.799479,31.992578,33.240885,0.348958
std,31.972618,115.244002,7.884160,11.760232,0.476951
min,0.000000,0.000000,0.000000,21.000000,0.000000
25%,99.000000,0.000000,27.300000,24.000000,0.000000
50%,117.000000,30.500000,32.000000,29.000000,0.000000
75%,140.250000,127.250000,36.600000,41.000000,1.000000
max,199.000000,846.000000,67.100000,81.000000,1.000000


In [ ]:
X = data[['Glucose', 'Insulin', 'BMI', 'Age']]
Y = data[['Outcome']]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
Y_train = pd.DataFrame(Y_train)
Y_test = pd.DataFrame(Y_test)

In [ ]:
naive_bayes_model = GaussianNB()

naive_bayes_model.fit(X_train, Y_train)

Y_pred_nb = naive_bayes_model.predict(X_test)

accuracy_nb = accuracy_score(Y_test, Y_pred_nb)
precision_nb = precision_score(Y_test, Y_pred_nb)
recall_nb = recall_score(Y_test, Y_pred_nb)
f1_nb = f1_score(Y_test, Y_pred_nb)

print(f"Naive Bayes Model - Accuracy: {accuracy_nb:.2f}")
print(f"Naive Bayes Model - Precision: {precision_nb:.2f}")
print(f"Naive Bayes Model - Recall: {recall_nb:.2f}")
print(f"Naive Bayes Model - F1 Score: {f1_nb:.2f}")

Naive Bayes Model - Accuracy: 0.79
Naive Bayes Model - Precision: 0.84
Naive Bayes Model - Recall: 0.58
Naive Bayes Model - F1 Score: 0.69


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
perceptron_model = Perceptron(max_iter=1000, tol=1e-3)
perceptron_model.fit(X_train, Y_train)
y_pred_perceptron = perceptron_model.predict(X_test)

accuracy_perceptron = accuracy_score(Y_test, y_pred_perceptron)
precision_perceptron = precision_score(Y_test, y_pred_perceptron)
recall_perceptron = recall_score(Y_test, y_pred_perceptron)
f1_perceptron = f1_score(Y_test, y_pred_perceptron)

print(f"Perceptron Model - Accuracy: {accuracy_perceptron:.2f}")
print(f"Perceptron Model - Precision: {precision_perceptron:.2f}")
print(f"Perceptron Model - Recall: {recall_perceptron:.2f}")
print(f"Perceptron Model - F1 Score: {f1_perceptron:.2f}")

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Perceptron Model - Accuracy: 0.60
Perceptron Model - Precision: 0.00
Perceptron Model - Recall: 0.00
Perceptron Model - F1 Score: 0.00


In [ ]:
class CustomPerceptron:
    def __init__(self, learning_rate=0.1, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.random.rand(n_features) * 0.1
        self.bias = 0

        for _ in range(self.n_iterations):
            errors = 0
            for idx, x_i in enumerate(X):
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_predicted = 1 if linear_output >= 0 else -1

                if y_predicted != y[idx]:
                    update = self.learning_rate * y[idx]
                    self.weights += update * x_i
                    self.bias += update
                    errors += 1

            if errors == 0:
                break

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        return np.where(linear_output >= 0, 1, -1)

Y_train_adjusted = np.where(Y_train.values.ravel() == 0, -1, 1)
Y_test_adjusted = np.where(Y_test.values.ravel() == 0, -1, 1)

custom_perceptron_model = CustomPerceptron(learning_rate=0.1, n_iterations=1000)
custom_perceptron_model.fit(X_train.values, Y_train_adjusted)
y_pred_custom = custom_perceptron_model.predict(X_test.values)

accuracy_custom = accuracy_score(Y_test_adjusted, y_pred_custom)
precision_custom = precision_score(Y_test_adjusted, y_pred_custom)
recall_custom = recall_score(Y_test_adjusted, y_pred_custom)
f1_custom = f1_score(Y_test_adjusted, y_pred_custom)

print(f"Custom Perceptron Model - Accuracy: {accuracy_custom:.2f}")
print(f"Custom Perceptron Model - Precision: {precision_custom:.2f}")
print(f"Custom Perceptron Model - Recall: {recall_custom:.2f}")
print(f"Custom Perceptron Model - F1 Score: {f1_custom:.2f}")

Custom Perceptron Model - Accuracy: 0.60
Custom Perceptron Model - Precision: 0.00
Custom Perceptron Model - Recall: 0.00
Custom Perceptron Model - F1 Score: 0.00


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
with open('naive_bayes_model.pkl', 'wb') as f:
    pickle.dump(naive_bayes_model, f)

with open('perceptron_model.pkl', 'wb') as f:
    pickle.dump(perceptron_model, f)